# Bank - Attritionanalysis

#### Daten einlesen

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path = "data/"
dataframes = []
for csv_file in os.listdir(path):
    if csv_file.endswith(".csv"):
        dataframes.append(pd.read_csv(path + csv_file, sep="|"))

df = pd.concat(dataframes, ignore_index=True)

print(df.shape)

### Kontrolle des zusammengefügten DataFrames

- Summe aller Zeilen der csv-Dateien
- Spaltenanzahl (Features)
- Ausgabe der Features

In [3]:
# path = "data/"
# sum_rows = 0
# for csv_file in os.listdir(path):
#     df = pd.read_csv(path + csv_file, sep="|")
#     sum_rows += df.shape[0]

# print(f"Summe der Zeilen aller csv-Dateien: {sum_rows}\nAnzahl der Features (Spalten): {df.shape[1]}\n{df.dtypes}")

### Erste Identifikation von Datenproblemen

#####Analyse der Spalten mit dem Datentyp object (string)

In [ ]:
def get_unique_objects():
    features = df.columns.values
    for feature in features:
        if df[feature].values.dtype not in [int, float]:
            df.drop_duplicates(inplace=True)
            print(f"{df[feature].name}\n{df[feature].unique()}")
          
#get_unique_objects()
df["gender"] = df["gender"].str.lower()
df["gender"] = df["gender"].replace("f", "female").replace("m", "male")
df["gender"] = df["gender"].fillna(df["gender"].mode()[0])
get_unique_objects()


##### Analyse der numerischen Spalten

In [ ]:
fig, ax = plt.subplots(figsize=(3, 6))

age = df["age"]

ax.boxplot([age])
ax.set_ylabel('Age (in years)')
ax.set_xticklabels(['Alter'])
ax.set_title('Box Plot: Age Feature')
plt.show()

print(f"Min Age: {df["age"].min()}\nMax Age: {df['age'].max()}\nAvg Age: {round(df['age'].mean(), 2)}")

###  [AT09] Identifikation von Ausreißern beim Kreditscore

Als Business möchte ich wissen, ob in dem Datensatz numerische Ausreißer existieren. Bitte wählen Sie eine geeignete Visualisierungsmethode, um den Kreditscore zu analysieren. Info: Sofern Ausreißer in der Spalte existieren, analysieren Sie, ob es sich um inhaltlich korrekte Werte handelt oder ob diese zu bereinigen sind (wenn ja, in welcher Form).

In [ ]:
fig, ax = plt.subplots(figsize=(3,6))

credit_score = df["credit_score"]

ax.boxplot(credit_score)
ax.set_title("Spalte credit_score")
ax.set_ylabel("Credit Score")
ax.set_xticklabels(["credit_score"])

Aus dem Boxplot erkennen wir 2 Outlier, einen Maximum-Outlier und einen Minimum-Outlier. Der Minimum-Outlier ist unter 0, das sieht inhaltlich falsch aus, aber der Maximum-Outlier könnte durchaus korrekt sein.

Schauen wir uns diese an.
Den min_credit_score ersetzen wir mit dem Median der credit_scores.

In [ ]:
min_credit_score = df["credit_score"].min()
max_credit_score = df["credit_score"].max()

df["credit_score"] = df["credit_score"].replace(min_credit_score, df["credit_score"].median())

#df.boxplot(column="credit_score", figsize=(3,6))

fig, ax = plt.subplots(figsize=(3,6))

credit_score = df["credit_score"]

ax.boxplot(credit_score)
ax.set_title("Spalte credit_score")
ax.set_ylabel("Credit Score")
ax.set_xticklabels(["credit_score"])

Wir sehen, der Minimum Ausreißer wurde entfernt.

### [AT10] Verteilung der Balance je Land

Als Business möchte ich wissen, wie die numerische Verteilung der Balance basierend auf dem Land des Kunden zu bewerten ist. Interpretieren Sie zusätzlich die Ergebnisse. Was kann ich als Business mit dem Ergebnis anfangen? Tipp: Verwenden Sie ein BoxPlot zur Visualisierung der Balance je Land.

Zuerst müssen wir wissen, welche Länder es gibt, danach erstellen wir länderspezifische Listen, die nur die Balance-Werte eines Landes enthalten.

In [8]:
balance = df["balance"]
countries = df["country"].unique()

countries
# Ausgabe: array(['France', 'Germany', 'Spain'], dtype=object)

fra_balance = df.loc[(df["country"] == "France"), "balance"]
ger_balance = df.loc[(df["country"] == "Germany"), "balance"]
spa_balance = df.loc[(df["country"] == "Spain"), "balance"]

Wir sehen, dass es keine Fehlerhaften werte wie "-" oder NaN in "country" gibt.
Nun visualisieren wir diese Ergebnisse und erstellen für jedes Land ein Boxplot.

In [ ]:
fig, ax = plt.subplots(dpi=200)

ax.boxplot([fra_balance, ger_balance, spa_balance])
ax.set_title("Verteilung der Balance je Land")
ax.set_xticklabels(["Frankreich", "Deutschland", "Spanien"])
ax.set_ylabel("Balance")
ax.axhline(ger_balance.median(), linestyle=":")
plt.show()

print(f"Datenpunkte für:\n- Deutschland: {len(ger_balance)}\n- Frankreich: {len(fra_balance)}\n- Spanien: {len(spa_balance)}")

Aus dem Plot können wir ableiten:
+ Deutsche haben bei dieser Bank im Durchschnitt mehr Geld auf ihrem Konto, als Kunden aus Frankreich oder Spanien.
+ In Frankreich und Spanien haben die Kunden etwa gleich viel Geld auf ihren Konten.
+ In Deutschland gibt es mehr Kunden, die besonders viel Geld auf ihrem Konto haben, als in Frankreich oder Spanien, weil die Daten stärker getreut sind.
+ Der deutsche Median-Kunde hat fast mehr Geld auf seinem Konto wie 75% der Kunden in Frankreich oder Spanien.

--> Zinsen auf das Konton könnten bei Deutschen Kunden für die Bank teurer sein. Angenommen in jedem Land sind die Zinsen bei 3% p.a., dann kann man Kosten begrenzen, indem man in Deutschland den Zinssatz auf 2,5% p.a. einstellt. Das ist dann ein Land, in dem sich das Risiko, Kunden zu verlieren lohnt, da dort die Zinsen auch teuerer sind.

Allgemein können mit den Infos des Plots Handlungsempfehlungen wie diese erstellt werden.

Man muss allerdings auf die Datenqualität aufpassen, da es für Frankreich doppelt so viele Datensätze gibt, wie für die beiden anderen Länder.

### [AT11] Verteilung der Balance zu Verfügbarkeit einer Kreditkarte

Als Business möchte ich wissen, wie sich die Balance der Kunden abhängig von der Existenz einer Kreditkarte unterscheidet. Wählen Sie eine geeignete Visualisierung und interpretieren Sie diese. (Info: Überprüfen Sie, ob leere Felder in den Attributen existieren. Wenn ja, schlagen Sie ein mögliches Vorgehen vor, um diese zu behandeln)

Identifizierung von der credit_card Spalte
In einem Zusammenhang bringen mit der Balance 

Könnte wieder durch Boxplots dargestellt werden, einen für Leute mit Karte und einen ohne Karte.

In [ ]:
credit_card = df["credit_card"]
balance = df["balance"]

credit_card.unique()
# array([ 1.,  0., nan])

credit_card = df["credit_card"].fillna(df["credit_card"].median())
credit_card.unique()
credit_card.shape
# array([1., 0.])

balance_with_credit_card = df.loc[credit_card == 1, "balance"]
balance_without_credit_card = df.loc[credit_card == 0, "balance"]

print(f"Anzahl Kunden mit credit_card: {len(balance_with_credit_card)}\nAnzahl Kunden ohne credit_card: {len(balance_without_credit_card)}")

empty_balance = df.loc[df["balance"].isna() == True, "balance"]
empty_balance
# Series([], Name: balance, dtype: float64)

zerobalance_with_credit_card = balance_with_credit_card.loc[balance == 0]
zerobalance_without_credit_card = balance_without_credit_card.loc[balance == 0]

zerobalance_with_credit_card_percentage = round((len(zerobalance_with_credit_card)/len(balance_with_credit_card))*100, 2)
zerobalance_without_credit_card_percentage = round((len(zerobalance_without_credit_card)/len(balance_without_credit_card))*100, 2)

print(f"Anzahl Kunden mit credit_card und balance 0: {zerobalance_with_credit_card_percentage}%\nAnzahl Kunden ohne credit_card und balance 0: {zerobalance_without_credit_card_percentage}%")

Es gab NaN-Werte bei der credit_card Spalte, aber nicht bei der Balance. Dort gibt es zwar 0, das ist aber ein realistischer Wert.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,9), dpi=200)

ax[0].hist([balance_with_credit_card], edgecolor="black")
ax[0].set_title("Mit Kreditkarte")
ax[0].set_ylabel("Balance")

ax[1].hist([balance_without_credit_card], edgecolor="black")
ax[1].set_title("Ohne Kreditkarte")
ax[1].set_ylabel("Balance")

plt.show()

+ Kunden mit Kreditkarte scheinen dazu zu neigen, eine höhere Balance zu haben.
+ Kunden ohne Kreditkarte tendieren demnach dazu, eine geringere Balance zu erhalten.

Bei Kunden mit Kreditkarten haben 36.73% eine Balance von 0 und bei Kunden ohne Kreditkarten haben 34.82% eine Balance von 0.
Wir können also sagen, dass pro Gruppe mind. ein Drittel der Leute kein Geld auf dem Konto hat.

### [AT19] Verteilung des Lands bei verlassenden Kunden zwischen 30 und 45

Als Business möchte ich wissen, wie die Verteilung des Lands (= country) bei Kunden zwischen 30 und 45 Jahren (= age) ausschaut, die die Bank verlassen (churn=1). Bitte wählen Sie eine geeignete Visualisierungsmethode, um die einzelnen Länder miteinander zu vergleichen.

In [12]:
customer_leaves = df.loc[df["churn"] == 1]
customer_stays = df.loc[df["churn"] == 0]

completeness_test = len(df) - (len(customer_leaves) + len(customer_stays))
# 0 --> complete set

age_filtered_customer_leaves = customer_leaves.loc[(customer_leaves["age"] >= 30) & (customer_leaves["age"] <= 45)]
age_filtered_customer_stays = customer_stays.loc[(customer_stays["age"] >= 30) & (customer_stays["age"] <= 45)]

fra_leaving_customers = age_filtered_customer_leaves.loc[(age_filtered_customer_leaves["country"] == "France"), "churn"]
ger_leaving_customers = age_filtered_customer_leaves.loc[(age_filtered_customer_leaves["country"] == "Germany"), "churn"]
spa_leaving_customers = age_filtered_customer_leaves.loc[(age_filtered_customer_leaves["country"] == "Spain"), "churn"]

fra_staying_customers = age_filtered_customer_stays.loc[(age_filtered_customer_stays["country"] == "France"), "churn"]
ger_staying_customers = age_filtered_customer_stays.loc[(age_filtered_customer_stays["country"] == "Germany"), "churn"]
spa_staying_customers = age_filtered_customer_stays.loc[(age_filtered_customer_stays["country"] == "Spain"), "churn"]

In [ ]:
countries = ('Frankreich', 'Deutschland', 'Spanien')
counts = {
    'Bleibt': [len(fra_staying_customers), len(ger_staying_customers), len(spa_staying_customers)],
    'Geht': [len(fra_leaving_customers), len(ger_leaving_customers), len(spa_leaving_customers)],
}
width = 0.8


fig, ax = plt.subplots(dpi=150)
bottom = np.zeros(3)

for churn, count in counts.items():
    p = ax.bar(countries, count, width, label=churn, bottom=bottom)
    bottom += count

    ax.bar_label(p, label_type='center')

ax.set_title('Anzahl bleibender und gehender Kunden pro Land')
ax.legend()

plt.show()

### [AT20] Erstellung zweier komibinierter Bar Plots

Als Business möchte ich wissen, wie die Verteilung der Zielvariable (= Churn) und der Verfügbarkeit einer Kreditkarte (= credit_card) ausschaut. Bitte wählen Sie einen kombinierten Bar Plot, bestehend aus einem vertikalen Bar Plot und einem horizontalen Bar Plot.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(3, 6))

churn_data = df.loc[df["churn"], "churn"]
credit_card.unique()
#array([1., 0.])

ax[0].boxplot(len(churn_data))
ax[1].boxplot(len(credit_card))